# Lecture 21 Classification IV: k-nearest neighbors 

# $k$-nearest neighbors ($k$NN) classifier

Setting: 

A simple classifier adopting the Bayesian thinking (a posteriori, without assuming a priori any model) is *kNN*. kNN can be thought as an approximation to the naive Bayes classifier. 

We will try to estimate the conditional probability of $y=j$ given training/testing sample $\mathbf{x}$ for each $j=1,\dots, K$, and then classify a given training/testing to the class with highest *estimated* probability.

Given a $k$ (different from the index for the class) and a test sample $\mathbf{x}$, the kNN classifier first identifies the neighbors $k$ points in the training data that are closest to $\mathbf{x}$, whose indices are represented by $\mathcal{N}$. It then estimates the conditional probability for class $j$ by computing the fraction of points in $\mathcal{N}$ whose label(s) actually equal $j$:

$$
P\big(y= j| \mathbf{x} \big)\approx  \frac{1}{k} \sum_{i\in \mathcal{N}} 1\{ y^{(i)} = j\}.
$$

Finally, kNN applies Bayesian rule and classifies the test sample $\mathbf{x}$ to the class with the largest estimated probability (the class with the most *votes* from those $k$-nearest neighbors). Despite the fact that it is a very simple approach, kNN can often produce classifiers that are surprisingly close to the optimal naive Bayes classifier.

## Implementation of $k$NN

Even the formulation above is pretty mathematical, the algorithm reads very simple as follows:

> Step 1: Assign a distance metric to the training samples $\{\mathbf{x}^{(i)} \}_{i=1}^N$ ($L^2$, $L^1$, $L^{\infty}$, etc), and choose the number of neighbors $k$.<br>
> Step 2: For the new data sample, take the $k$ nearest neighbors of this sample in $\{\mathbf{x}^{(i)} \}_{i=1}^N$, according to the distance metric.<br>
> Step 3: Among these $k$ neighbors, count the number of data samples in each class.<br>
> Step 4: Assign this new data sample to the class where the most neighbors are counted.

Remark: To avoid tie situations, we note that $k$ is usually odd.

## A famous example: classify flowers using the lengths of petals/sepals

The data set we are using is the [Iris Flower Dataset (IFD)](https://www.kaggle.com/uciml/iris).Here we will use `scikit-learn`'s dataset module to import it.

IFD was first introduced in 1936 by the famous statistician Ronald Fisher and consists of 50 observations from each of three species of Iris (Iris setosa, Iris virginica and Iris versicolor). Four features were measured from each sample: the length and the width of the sepals and petals. Our goal is to train a kNN algorithm to be able to distinguish the species from one another given the measurements of the 4 features.

In the in-class presentation, we will use `scikit-learn`'s built-in kNN classifier. A full implementation using `numpy` is the in the end of this notebook for you to read.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# if we load the raw Iris.csv, we have to pre-process the data
iris_data = pd.read_csv('Iris.csv')
iris_data.head()

In [ ]:
# First we drop the Id column using the drop function in pandas Dataframe class
# then visualize the dataset using seaborn.
sns.pairplot(iris_data.drop(labels = ['Id'], axis=1), hue='Species')  # dropping the Id column
plt.show()

In [ ]:
sns.set()
sns.pairplot(iris_data.drop(labels = ['Id'], axis=1), diag_kind='hist', hue='Species')  # dropping the Id column
plt.show()

## Using scikit-learn

Below we load the data from `scikit-learn`, since the label is already pre-processed as numbers.

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris_data_proc = load_iris()
iris_data_proc.keys()

In [ ]:
X, y = iris_data_proc.data, iris_data_proc.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

## kNN from scikit-learn

In [ ]:
# loading library
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# instantiate learning model (k = 5)
iris_knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
# fitting the model
iris_knn.fit(X_train, y_train)

In [ ]:
# predict the response
y_pred = iris_knn.predict(X_test)

In [ ]:
y_pred[:10]

In [ ]:
# evaluate accuracy
print("The testing accuracy is: ", np.mean(y_test == y_pred))

# In-class exercise:

* Read the [reference of `KNeighborsClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).
* Try different number of `n_neighbors` (odd or even numbers), what have you observed.
* There is an option `weights ='distance'` in the `KNeighborsClassifier` class, try it.

# Implement the $k$NN algorithm

Following the main working pipeline we have seen in regression, we need to implement the following three components:

* Model: choose the distance metric we want to use, choose $k$
* Training: there is actually no training since there is no model, just store the data.
* Testing/Cross-validate: compute the distance between a testing sample and all training samples. Sorting and get the $k$ nearest neighbors, and perform a majority to determine the class.

### Distance choosing
The most commonly used is the Euclidean distance ($L^2$-distance):
$$
d(\mathbf{x}, \mathbf{x}') = \sqrt{(x_1 - x'_1 )^2 + (x_2 - x'_2 )^2 + \dots + (x_n - x'_n )^2},
$$
for other distances, please refer to [the DistanceMetric class in scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html). The next popular one is Manhattan distance ($L^1$-distance), which performs much better than $L^2$ in high dimension (the data point has many features).

First we notice that sorting the $L^2$ distance $d(\mathbf{x}, \mathbf{x}')$ is like sorting the distance squared, so we do not have to take square root just to save some time.

In [ ]:
# a simple kNN implementation
M = len(y_test)
N = len(y_train)
k = 5 # no. of neighbors to a testing sample
y_pred = np.zeros(M, dtype = int) # the prediction of the labels

In [ ]:
for j in range(M):
    dist_j = np.zeros(N)
    # initialization: dist_j stores the distances of j-th testing sample to all training samples
    for i in range(N):
        dist_j[i] = np.sum((X_test[j,:] - X_train[i,:])**2)
        # dist_j[i] is the distance of j-th testing sample to i-th training sample
    idx_knn = np.argsort(dist_j)[:k] # return the indices of k nearest neighbors
    # X_train[idx_knn[0], :] will be the training sample that is closest to j-th testing sample
    label_neighbor = y_train[idx_knn]
    # y_train[idx_knn] is the labels of these k nearest neighbor
    label_neighbor, count_label = np.unique(label_neighbor, return_counts=True)
    # count_label: how many time a certain labeled samples appear as in the k nearest neighbors
    y_pred[j] = label_neighbor[np.argmax(count_label)]
    # np.argmax(count_label) returns the index of the label appearing 

# evaluate accuracy
print("The testing accuracy is: ", np.mean(y_test == y_pred))

### Usages of sort, argsort, and unique in numpy

In [ ]:
# examples of argsort vs sort
arr = np.array([1, 0, 5, 19, 12, 3])

In [ ]:
np.sort(arr) # returns the sorted array

In [ ]:
np.argsort(arr) # returns the indices of the sorted entry
# the first entry 1 means element 1 in arr is 0, the smallest entry
# the second entry 0 means element 0 in arr is 1, the second smallest entry
# the j-th entry in the argsort means element j in arr is the j-th smallest entry

In [ ]:
count_label

In [ ]:
# an example of np.unique
label_neighbor, count_label = np.unique([2, 1, 1, 1, 1, 1, 0, 0], return_counts=True)
print(label_neighbor) # the unique labels in the array
print(count_label) # how many times each labels appearing in the array

In [ ]:
np.argmax(count_label) # the most frequent label

## Reading: Vectorization of kNN
A common way to implement $k$NN is to run a `for` loop for all testing samples. In $i$-th iteration of the loop, the distances between the $i$-th testing sample and all training samples are computed and sorted, so that we can choose the $k$-smallest distance neighbors. This is highly un-vectorized, and not recommended for large dataset.

Instead we consider the problem in bulk, suppose we have $n$ features in our dataset: we need to find the $L^2$-distance squared between the a set of test vectors (representing the testing samples), stored in array `X_test` of shape `(M,n)`, and a set of training vectors (representing the training samples), held in a matrix `X_train` of shape `(N,n)`. Our goal is to create a distance matrix `D` of shape `(M,N)` that stores the $L^2$-distance squared from every test vector to every training vector, for example `D[j,i]` shall return the distance squared between the $j$-th testing sample and the $i$-th training sample, and `D[j,:]` the $j$-th row of `D` stores the distance squared between the $j$-th testing sample and all training samples.

To do this in a vectorized way, a simple for $L^2$ distance is using $|x-x'|^2 = x^2 -2xx' + x'^2$. A vectorized implementation with no `for` loop iterating around all testing samples is in the following cell, for moderate dataset like MNIST, the performance is about several hundred times faster than using `for` loop.

In [ ]:
# Dist[j,i] is the np.sum((X_test[j,:] - X_train[i,:])**2)
Dist = -2 * np.dot(X_test, X_train.T) + np.sum(X_train**2, axis=1) + np.sum(X_test**2, axis=1).reshape(-1,1)
# try testing each term's shape in a new cell and ponder about this implementation
# notice np.sum(X_test**2, axis=1).reshape(-1,1) is the same with np.sum(X_test**2, axis=1)[:, np.newaxis]

In [ ]:
# now the implementation is almost the same with above
k = 5 # k nearest neighbors
K = 3 # K classes
idx_knn = Dist.argsort(axis = 1)[:,:k] # sort by columns for each row, then return the first k columns
knn_label = y_train[idx_knn]
# the vectorized version takes some tricks
count_label = np.zeros((len(y_test), K), dtype = int)
for j in range(K): # a small for loop iterating on classes
    count_label[:,j] = np.sum((knn_label==j), axis=1)
y_pred = np.argmax(count_label, axis=1)

# evaluate accuracy
print("The testing accuracy is: ", np.mean(y_test == y_pred))

Some cells below are for you to figure out what happened above.

In [ ]:
np.sum(X_test**2, axis=1).reshape(-1,1).shape

In [ ]:
np.dot(X_test, X_train.T).shape

In [ ]:
np.sum(X_train**2, axis=1).shape

In [ ]:
y_train[idx_knn][:10,:]

In [ ]:
count_label[:10,]